# The regression problem

In regression, one predicts a response/dependent variable from known explanatory variables/independent variables/features/responses. Some applications include:
* interpolation/extrapolation/prediction/estimation of the response
* parameter estimation (including uncertainty)
* interpretation, e.g. determination the most influential explanatory variables

Linear regression is linear in the parameters, but it can be nonlinear in explanatory variables. As an example, when measurements with quadratic behavior are fitted with a straight line, we find that the residuals are highly correlated. In this case, we can create a second feature $x^2$ and fit a linear hyperplane to the features $(x, x^2)$ with coefficients $\beta_1, \beta_2$.

<img src="img/quadratic-regression.png" alt="Drawing" style="width: 600px;"/>

## Model and assumptions for linear regression

The linear regression model is given by:

$$ y = \beta^T x + \epsilon(x) $$

where $x$ is deterministic (i.e. given) and exact (no measurement errors in explanatory variables), $\beta$ is the (true) parameter that is deterministic, $\epsilon(x)$ is random noise, and as a result $y$ is also a random variable. It should be noted that the residuals of LR/OLS (ordinary least squares) are evaluated only in $y$; c.f. residuals of TLS (total least squares) that minimize the perpendicular distance to the fitted line. The assumptions are:
1. The explanatory variables are exact
2. $\mathbb{E}_\epsilon[\epsilon(x)] = 0$
3. $\mathrm{Var}[\epsilon(x)] = \mathrm{const}$, independent of x (homoscedasticity)
4. $\epsilon(x)$ are iid (independent and identically distributed)

Note the shape of the matrices: $y$ is $(1 \times n)$, $x$ is $(p \times n)$, $\epsilon$ is $(1 \times n)$, and $\beta^T$ is $(1 \times p)$. We want to find parameters $\beta$ that minimize the sum of squared residuals (SSQ) defined as:

$$ \mathrm{SSQ} = (Y - \beta^T X) (Y - \beta^T X)^T $$

where $Y$ is the vector of measured responses, the SSQ is a scalar number and the two terms have shapes $1 \times n$ and $n \times 1$ respectively. We therefore take the derivative of the SSQ w.r.t. the parameters:

$$ \frac{\partial \mathrm{SSQ}}{\partial \beta} = -2 Y X^T + 2 \beta^T X X^T $$

Setting this to zero, we get the so-called **normal equation**:

$$ XX^T \beta = XY^T $$

from which we get an expression for the parameters:

$$ \beta = (XX^T)^{-1} XY^T $$

Shapes of each term are $p \times p$, $p \times n$, and $n \times 1$ respectively. A potential problem is when the matrix $XX^T$ is close to being irregular (e.g. when there are too few points or too many dimensions). When the measurements are correlated, we get a single tiny eigenvalue, thus the inverse would not be well-defined. In particular, if the measurements lie on a horizontal line in the feature space, the fitted coefficients will have low uncertainty along that horizontal dimension but high uncertainty along the dimension with ill-defined slope (the vertical dimension). This well-posedness of the problem depends only on the explanatory variables but not the response! Additional comments:
* The parameter estimate is a linear function of Y: the estimated parameters are also random variables.
* Prediction $\hat{Y} = \beta^T X = Y X^T (XX^T)^{-1} X = Y H $, where $H = X^T (XX^T)^{-1} X$ is often called the "hat matrix".
* Residuals are $\epsilon = Y - \hat{Y} = Y (I - H) $.

## Distribution of the learned coefficients

So far, we have found an estimate for the mean of the distribution of the random variable $\beta$. Next consider

$$ \begin{split}
\mathrm{Cov}_\epsilon (\hat{\beta}) & = \mathrm{Cov}_\epsilon ((XX^T)^{-1} XY^T) \\
    & = \mathrm{Cov}_\epsilon ((XX^T)^{-1} X(X^T \beta^* + \epsilon^T)) 
 \end{split}$$
 
where $\beta^*$ is the vector of true parameters. Now, note that the first term in the summation is deterministic thus can be omitted:

$$ \begin{split}
\mathrm{Cov}_\epsilon (\hat{\beta}) & = \mathrm{Cov}_\epsilon ((XX^T)^{-1} X\epsilon^T) \\
    & = (XX^T)^{-1} X \, \mathrm{Cov} \, (\epsilon^T)X^T(XX^T)^{-1}
 \end{split}$$
 
where we have made use of $ \mathrm{Cov}(A\epsilon^T) = A \mathrm{Cov}(\epsilon^T) A^T$. Now, using the fact that $\epsilon$ is iid so that $ \mathrm{Cov} (\epsilon) = \sigma^2 I$, we get

$$ \mathrm{Cov}(\hat{\beta}) = \sigma^2 (XX^T)^{-1} $$

This result holds true for any distributions of residuals $\epsilon$ so long as $\epsilon$ is iid with $\mathbb{E}(\epsilon) = 0$. If in addition we assume $ \epsilon ~ N(0,\sigma^2)$ then $\hat{\beta}$ is also normally distributed. We assume here that the estimate is bias-free: the distributions are centered on the true parameters $\beta^*$. One disadvantage of linear regression is that the estimate $\hat{\beta}$ is sensitive to outliers, e.g. a single outlier in the measurements of $Y$ can drastically change the parameter estimates.

## Bias-variance decomposition of mean squared error

In practice, to get the estimated parameters $\hat{\beta}$, one solves the normal equation directly instead of taking the inverse of $(XX^T)$. Now, we say that the matrix is irregular if the inverse does not exist, and likewise we say that the matrix is regular if the inverse exists. Often, $XX^T$ is nearly irregular. Possible reasons are:
+ number of features are larger than number of samples (p > N)
+ collinearity in the observations

In particular, OLS has zero bias: as $N \rightarrow \infty$, we are going to recover the true parameter values. *But* OLS suffers from high variance. Let's assume that the parameter is a scalar; the mean squared error of the estimate *w.r.t.* the true parameter is:

$$ \begin{split}
\mathbb{E}[(\hat{\theta} - \theta)^2] & = \mathbb{E}[(\hat{\theta} - \mathbb{E}[\hat{\theta}] + \mathbb{E}[\hat{\theta}] - \theta)^2] \\
   & = \mathbb{E}[(\hat{\theta} - \mathbb{E}[\hat{\theta}])^2] + \mathbb{E}[(\mathbb{E}[\hat{\theta}] - \theta)^2] + 2 \mathbb{E}[(\hat{\theta} - \mathbb{E}[\hat{\theta}])(\mathbb{E}[\hat{\theta}] - \theta)]\\
   & = \mathbb{E}[(\hat{\theta} - \mathbb{E}[\hat{\theta}])^2] + (\mathbb{E}[\hat{\theta}] - \theta)^2
\end{split}$$

The first term is the **variance** (i.e. the mean squared deviation of the parameter estimate from the expected parameter estimate); the second term is the squared **bias** (i.e. how much the expected estimate deviates from the true parameter value).

<img src="img/empirical-mse.png" alt="Drawing" style="width: 800px;"/>

General idea of regularization: reduce variance by limiting the flexibility of the model + choose a sweet spot by cross-validation. Three concrete methods will be discussed below.

### PCR (Principal Component Regression)

* project data to r-dim subspace (r < p)
* then do OLS (ordinary least squares) in that subspace

Here we keep only the first few eigenvalues (out of total p) of $XX^T$, as small eigenvalues give unstable results (high uncertainty in the corresponding direction of the eigenvector).

### Ridge Regression (1970s)

$$\min_\beta || Y - \beta^T X||^2_2 + \lambda ||\beta||^2_2 $$

The first term is the OLS, second term bias solution towards modest slopes. $\lambda$ is the regularization strength. This can be solved in closed form by taking the partial derivative w.r.t. $\beta$:

$$ \frac{\partial}{\partial \beta} \left( YY^T - 2 Y X^T \beta + \beta^T XX^T \beta + \lambda \beta^T \beta \right) \\
= -2 YX^T + 2 \beta^T XX^T + 2 \lambda \beta^T$$

Setting this expression to zero, we get

$$ \beta^T (XX^T + \lambda I) = Y X^T $$

or an explicit expression for the parameter estimate

$$ \hat{\beta} = (XX^T + \lambda I)^{-1} X Y^T $$

Note the additional ridge regularization term (the identity matrix multiplied by a constant). While PCR completely ignores small eigenvalues, ridge stabilizes the inverse operation by increasing all eigenvalues of $XX^T$ by a constant value. In other words, ridge penalty artificially inflates the covariance matrix $XX^T$ in all dimensions, i.e. it spreads out the data in all dimensions. As one increases the regularization term, eventually all coefficients would become zero, but it can't make the problem sparser by selectively making certain coefficients to be close to zero (useful for feature selection).

### LASSO regression (~1996)

Our aim is to be able to make the problem sparser (i.e. setting some coefficients to zero). We can introduce

$$ \min_\beta || Y - \beta^T X||^2_2 + \lambda || \beta||_0$$

where the L0-norm counts the number of nonzero components in the vector. For instance, when $p = 2$, we have to compute three solutions:
1. $\beta_1 = 0, \beta_2 \neq 0$
2. $\beta_1 \neq 0, \beta_2 = 0$
3. $\beta_1 \neq 0, \beta_2 \neq 0$

Due to the combinatorial explosion of number of possible solutions to try, we need something more tractable than the L0-norm. Hence we introduce the $L_p$-norm (the $p$ here is not related to the dimension of the features, so we use $l$ instead in the formula below to avoid confusion):

<img src="img/equicountour.png" alt="Drawing" style="width: 400px;"/>

The equicountour lines (the surfaces where the values of the loss are equal) for intermediate values of $l$ can be interpolated from the ones shown above. When $l \geq 1$, however, the regularizers are convex. We therefore use the L1-norm for LASSO regression:

$$ \min_\beta \mathrm{SSQ}(\beta | X, Y) = \min_\beta (Y - \beta^T X) (Y - \beta^T X)^T + \lambda || \beta||_1 $$

We can also write the regularization term as follows

$$ || \beta||_1 = a(\beta)^T \beta$$

where $a(\beta) \in \{-1, +1\}^p$ by the definition of the L1-norm. Here, it is clear that $||\beta||_1$ is just a plane in each of the quadrants. Therefore, the SSQ is still a parabola because $ YY^T + \beta^T XX^T \beta - 2 \beta^T XY^T$ is a parabola and $a(\beta)^T \beta$ are different planes in each orthant (=generalization of quadrant to higher dimensions). In other words, the objective is just a parabola in each orthant. Moreover, the parabolae match up at the orthant boundaries since the function $||\beta||_1$ is continous across orthants. Thanks to convexity, there is a single minimum.

<img src="img/lasso-energy.png" alt="Drawing" style="width: 800px;"/>

Let's now compare the solutions of LASSO and ridge by using the equivalent constraint formulations to see why we get sparse solutions in the case of LASSO but not ridge.

<img src="img/lasso-vs-ridge.png" alt="Drawing" style="width: 800px;"/>

Moreover, the space of sparse solutions becomes larger as $\kappa$ is reduced (or equivalently if $\lambda$ is increased):

<img src="img/space-sparse.png" alt="Drawing" style="width: 800px;"/>

Now, as an example of linear regression, consider computed tomography:
* measurements $y$ are the detected absorption with shape (1 x n), where n is the number of rays
* unknowns $\hat{\beta}$ are the intensities of pixels with shape (p x 1), where p is the number of pixels

<img src="img/tomography.png" alt="Drawing" style="width: 800px;"/>